In [8]:
import json
import gzip
from collections import deque

In [9]:
with gzip.open("data_json/tree.json.gz", "rt", encoding="utf-8") as f:
    data = json.load(f)
print(data.keys())
with gzip.open("data_json/root-sequence.json.gz", "rt", encoding="utf-8") as f:
    root_seq = json.load(f)
print(set(root_seq.keys()))

dict_keys(['meta', 'tree', 'version'])
{'nuc', 'M', 'E', 'ORF3a', 'ORF7a', 'ORF8', 'ORF9b', 'N', 'ORF7b', 'S', 'ORF6', 'ORF1b', 'ORF1a'}


In [32]:
tree = data['tree']
tree['node_attrs']

{'clade_membership': {'value': 'unassigned'},
 'div': 0,
 'division': {'confidence': {'California': 0.003207482209756541,
   'China': 0.47525589120801487,
   'Hubei': 0.470945649102106},
  'entropy': 1.1246196751582915,
  'value': 'China'},
 'num_date': {'confidence': [2019.9390194860111, 2019.9748082943493],
  'value': 2019.9417641417247}}

In [29]:
dates = []
q = deque([tree])
visited = set()
counter = 0
parents = []
ancestral_muts = []
tip_muts = []
MA_muts = []
clades = set()
while q:
    # process node
    node = q.popleft()
    visited.add(node["name"])
    dates.append(node["node_attrs"]["num_date"]["value"])


    if node["branch_attrs"]["mutations"]:
        if "NODE" in node["name"]:
            ancestral_muts.append(node["branch_attrs"]["mutations"])
        else:
            if 'MA' in node['name']:
                MA_muts.append(node["branch_attrs"]["mutations"])
            tip_muts.append(node["branch_attrs"]["mutations"])
    

        
    clades.add(node['node_attrs']['clade_membership']['value'])
    
    counter += 1

    children = []
    if "children" in node:
        children = node["children"]

    if children:
        parents.append(node["name"])
    for child in children:
        if child["name"] in visited:
            continue
        q.append(child)

print(f"Visited {counter} sequences")

Visited 14761 sequences


In [12]:
inferred_ancestors = [name for name in visited if "NODE" in name]
tip_sequences = [name for name in visited if "NODE" not in name]

print(f"{len(inferred_ancestors)} inferred ancestors")
print(inferred_ancestors[:5])

print(f"{len(tip_sequences)} observed tip sequences")
print(tip_sequences[:5])

ma_tip_seqs = [name for name in tip_sequences if "MA" in name]
len(ma_tip_seqs)
ma_tip_seqs[:10]

6857 inferred ancestors
['NODE_0007630', 'NODE_0007268', 'NODE_0006769', 'NODE_0001670', 'NODE_0007440']
7904 observed tip sequences
['USA/WA-CDC-UW21080367447/2021', 'USA/GA-GPHL-0727/2021', 'USA/MA-CDC-ASC210118487/2021', 'AUS/VIC1143/2020', 'USA/MA-CDCBI-CRSP_XWPAXLKLXAIWLKJD/2021']


['USA/MA-CDC-ASC210118487/2021',
 'USA/MA-CDCBI-CRSP_XWPAXLKLXAIWLKJD/2021',
 'USA/MA-MASPHL-04267/2021',
 'USA/MA-CDCBI-CRSP_XYDIOFJTIEG7J2MC/2021',
 'USA/MA-CDC-STM-000031966/2021',
 'USA/MA-CDCBI-CRSP_5J6ZDGPBUXA523GS/2021',
 'USA/MA-CDCBI-CRSP_4WHVSEDQ6XIYDFV2/2021',
 'USA/MA-CDCBI-CRSP_23GLYXQ52M4674AK/2021',
 'USA/MA-CDCBI-CRSP_SFENDXKHK2XR2MV2/2021',
 'USA/MA-CDCBI-CRSP_IBECQ2SBHNGI5TMT/2021']

In [13]:
# sanity check
# none of the tip nodes are parents
set(parents) == set(inferred_ancestors)

True

In [14]:
class GenomeNode:
    def __init__(self, node):
        self.name = node["name"]
        self.branch_attrs = node["branch_attrs"]
        self.node_attrs = node["node_attrs"]
        
        self.extract_node_info()
        self.extract_branch_info()


        self.children = node.get("children")
        self.children = [GenomeNode(child) for child in children]



    def extract_node_info(self):
        self.clade = self.node_attrs["clade_membership"]
        self.divergence = self.node_attrs["div"]
        self.ancestral_location = self.node_attrs["division"]
        self.collection_date = self.node_attrs["num_date"]


    def extract_branch_info(self):
        mutations = self.branch_attrs['mutations']


    def is_collected_tip(self) -> bool:
        return "NODE" not in self.name

    def __str__(self):
        return f"Genome sequence: {self.name}"

In [15]:
# look at root node
name = tree['name']
branch_attrs = tree['branch_attrs']
node_attrs = tree['node_attrs']

child_1 = tree['children'][1]['children'][0]
# child_1['branch_attrs']['mutations'].keys()

# branch_attrs['mutations'].keys()

In [16]:
all_keys = set()

all_nuc_mut_len = []
all_muts_len = []
no_s_mut = 0
for mut in ancestral_muts + tip_muts:
    if "S" not in mut:
        no_s_mut += 1
    len_nuc_mut = len(mut["nuc"])
    len_all_muts = 0
    for key, value in mut.items():
        if key == "nuc":
            continue
        len_all_muts += len(mut[key])

    all_nuc_mut_len.append(len_nuc_mut)
    all_muts_len.append(len_all_muts)


print(f"{100 * len(MA_muts) / len(ma_tip_seqs)}% of tip sequences from Massachussets have mutations recorded")

print(
    f"{100 * len(tip_muts) / len(tip_sequences)}% of tip sequences have mutations recorded"
)

print(
    f"{100 * len(tip_muts)/len(ancestral_muts + tip_muts)}% of mutations belong to tip nodes"
)

print(
    f"{100 * len(ancestral_muts) / len(inferred_ancestors)}% of ancestral sequences have mutations recorded"
)

print(
    f"{100 * len(ancestral_muts)/len(ancestral_muts + tip_muts)}% of mutations belong to ancestral nodes"
)

print(len(ancestral_muts + tip_muts))
# all_keys == {
#     "ORF7a",
#     "ORF8",
#     "M",
#     "nuc",
#     "ORF9b",
#     "E",
#     "N",
#     "ORF1b",
#     "ORF1a",
#     "S",
#     "ORF3a",
#     "ORF7b",
#     "ORF6",
# }

38.9171974522293% of tip sequences from Massachussets have mutations recorded
49.3421052631579% of tip sequences have mutations recorded
66.99879745748153% of mutations belong to tip nodes
28.01516698264547% of ancestral sequences have mutations recorded
33.00120254251847% of mutations belong to ancestral nodes
5821


In [23]:
tip_muts[1010]

{'E': ['F56L'], 'nuc': ['C7165T', 'C8389T', 'T26410C']}

In [30]:
clades

{'19B',
 '20A/N.194L',
 '20B/S.484K',
 '20D',
 '20F',
 '20G',
 '20J/501Y.V3',
 'unassigned'}